In [1]:
#default_exp segmentation.metrics

In [2]:
%autosave 60 
import fastai; print(fastai.__version__)

Autosaving every 60 seconds
1.0.58.dev0


In [3]:
#export
from fastai.vision import *
from fastai.metrics import foreground_acc, dice

In [4]:
#export
_all_ = ["foreground_acc", "dice"]

### test

In [5]:
from local.test import *

In [6]:
_input_softmax = tensor([[
    [
    [0,0,1],
    [0,0,1],
    [1,1,1]
],
    [
    [1,1,0],
    [1,1,0],
    [0,0,0]
]
]]).float()

_input_sigmoid = tensor([[
    [
    [2,2,-2],
    [2,-2,1],
    [-2,1,1]
],
    [
    [1,1,-1],
    [1,-1,2],
    [-1,2,2]
]
]]).float()

_input_sigmoid2 = tensor([[[[-10,-10],[-10,-10]],[[10,10],[10,10]]]]).float()

_target_softmax = tensor([[[
    [0,0,0],
    [0,0,1],
    [0,1,1]
]]])

_target_sigmoid = tensor([[[
    [1,1,0],
    [1,0,2],
    [0,2,2]
]]])

_target_sigmoid2 = tensor([[[[0,0],[0,0]]]])

_input_sigmoid = torch.cat([_input_sigmoid for i in range(5)])
_input_softmax = torch.cat([_input_softmax  for i in range(5)])
_target_sigmoid = torch.cat([_target_sigmoid  for i in range(5)])
_target_softmax = torch.cat([_target_softmax  for i in range(5)])

_input_softmax.size(), _input_sigmoid.size(), _target_sigmoid.size(), _target_softmax.size()

(torch.Size([5, 2, 3, 3]),
 torch.Size([5, 2, 3, 3]),
 torch.Size([5, 1, 3, 3]),
 torch.Size([5, 1, 3, 3]))

In [7]:
test_eq(dice(_input_softmax, _target_softmax, eps=1e-8), 0)

### `iou`

In [8]:
#export
def iou(input: torch.Tensor, targs: torch.Tensor, **kwargs)->Rank0Tensor:
    "Binary IOU"
    return dice(input, targs, iou=True, **kwargs)

In [9]:
test_eq(iou(_input_softmax, _target_softmax, eps=1e-8), 0)

### `_dice`

$$
DSC =\frac{2 |X \cap Y|}{|X|+ |Y|} 
\hspace{20pt} 
IOU =\frac{|X \cap Y|}{|X \cup Y|}
$$

In [10]:
#export
def _dice(input:Tensor, targs:Tensor, iou:bool=False, 
          eps:float=1e-8, reduce:bool=True)->Rank0Tensor:
    "Dice coefficient metric for probas and binary target."
    n = targs.shape[0]
    input = input.view(n,-1).float()
    targs = targs.view(n,-1).float()
    intersect = (input * targs).sum(dim=1).float()
    union = (input+targs).sum(dim=1).float()
    if not iou: l = 2. * intersect / union
    else: l = intersect / (union-intersect+eps) 
    l[union==0.] = 1.    
    if reduce: return l.mean()
    else: return l

In [11]:
assert _dice(tensor([[1]]), tensor([[1]])).item() == 1
assert _dice(tensor([[0]]), tensor([[0]])).item() == 1
assert _dice(tensor([[1]]), tensor([[0]])).item() == 0
assert _dice(tensor([[0]]), tensor([[1]])).item() == 0

### `mean_dice`

In [29]:
#export
def mean_dice(input, target, c, macro=True, **kwargs):    
    "macro: mean of class dice, micro: mean of mean dice image (excluding class:0)"
    if macro:
        res = [_dice(input==ci, target==ci, reduce=True, **kwargs) for ci in range(1, c)]
        return torch.mean(tensor(res).to(input.device))    
    else:
        res = [_dice(input==ci, target==ci, reduce=False, **kwargs) for ci in range(1, c)]
        return torch.stack(res).to(input.device).mean(0).mean()    

In [30]:
x = tensor([[[0,1,2],[0,1,2],[0,1,2]]])
y = tensor([[[1,1,0],[1,0,2],[0,2,2]]])
test_eq(mean_dice(x,y,c=3), 0.5)
test_eq(mean_dice(x,y,c=3,iou=True), 0.35)

### `_to_sigmoid_input`

In [31]:
#export
def _to_sigmoid_input(logits, threshold=0.5, exclude_void=True):
    "convert logits to preds with sigmoid and thresh, assumes void is excluded"
    sigmoid_input = logits.sigmoid()
    thresholded_input = sigmoid_input > threshold
    _, indices = torch.max(sigmoid_input, dim=1)
    if exclude_void: indices += 1
    values, _ = torch.max(thresholded_input, dim=1)
    preds = (values.float()*indices.float())
    return preds

In [32]:
assert _to_sigmoid_input(_input_sigmoid, threshold=1).view(-1).sum(0) == 0
test_eq(_to_sigmoid_input(_input_sigmoid, threshold=0)[0],
        tensor([[1., 1., 2.], [1., 2., 2.],[2., 2., 2.]]))
test_eq(_to_sigmoid_input(_input_sigmoid, threshold=0.5)[0],
        tensor([[1., 1., 0.], [1., 0., 2.], [0., 2., 2.]]))

### `sigmoid_mean_dice`

In [33]:
#export 
def sigmoid_mean_dice(input:Tensor, target:Tensor, threshold:float=0.5, **kwargs)->Rank0Tensor:
    "mean_dice with sigmoid output which doesn't predict background"
    c = input.size(1)+1
    input = _to_sigmoid_input(input, threshold)    
    return mean_dice(input, target, c, **kwargs)

In [34]:
test_close(sigmoid_mean_dice(_input_sigmoid, _target_sigmoid, 
                            macro=True, threshold=0).item(), 0.833, eps=1e-3)
assert sigmoid_mean_dice(_input_sigmoid, _target_sigmoid, macro=True)
assert sigmoid_mean_dice(_input_sigmoid, _target_sigmoid, macro=False)
assert sigmoid_mean_dice(_input_sigmoid2, _target_sigmoid2, macro=True) == 0.5

### `softmax_mean_dice`

In [35]:
#export 
def softmax_mean_dice(input:Tensor, target:Tensor, **kwargs)->Rank0Tensor:
    "mean_dice with softmax output which includes background prediction"
    c = input.size(1)
    input = input.argmax(1)
    return mean_dice(input, target, c, **kwargs)

In [36]:
test_eq(softmax_mean_dice(_input_sigmoid2, _target_sigmoid2), 0.)

### export

In [37]:
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_script.ipynb.
Converted 02_scheduler.ipynb.
Converted 03_callbacks.ipynb.
Converted 10_segmentation_dataset.ipynb.
Converted 11_segmentation_losses_mulitlabel.ipynb.
Converted 11b_segmentation_losses_binary.ipynb.
Converted 12_segmentation_metrics.ipynb.
Converted 13_segmentation_models.ipynb.
Converted segmentation_training.ipynb.


### fin